In [1]:
import torch
from modules.modules import HyperNet, MainNet
import ikpy.chain
import numpy as np


In [2]:
# Runtime Configuration
class Config:
    chain_path='assets/UR5/urdf/ur5_robot.urdf'
    train_data_path='data/ur5/ur5_train_data.csv'
    test_data_path='data/ur5/ur5_test_data.csv'
    num_joints=6
    lr=0.001
    num_epochs=200
    num_solutions_validation=10
    batch_size=1024
    early_stopping_epochs=30
    grad_clip=1
    embedding_dim=128
    hypernet_input_dim=6
    hypernet_hidden_size=1024
    hypernet_num_hidden_layers=3
    jointnet_hidden_size=256
    num_gaussians=50
    exp_dir='runs/exp_12'
    jointnet_output_dim=150
    jointnet_output_dim = 2 if num_gaussians == 1 else num_gaussians * 2 + num_gaussians

cfg = Config()


In [3]:
# Initialize HyperNet and MainNet
hypernet = HyperNet(cfg)
mainnet = MainNet(cfg)


In [4]:
# Load the best model weights into HyperNet
model_path = "runs/exp_11/best_model.pt"
hypernet.load_state_dict(torch.load(model_path))
hypernet.eval()  # Set the model to evaluation mode

C:\Users\SAE\AppData\Local\Temp\ipykernel_26448\629899569.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hypernet.load_state_dict(torch.load(model_path))


HyperNet(
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=1024, bias=True)
    (1-2): 2 x Linear(in_features=1024, out_features=1024, bias=True)
  )
  (out): Linear(in_features=1024, out_features=128, bias=True)
  (projection): MultiHeadLinearProjection(
    (linears): ModuleList(
      (0-1): 2 x ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=256, bias=True)
        )
      )
      (2): ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=38400, bias=True)
        )
      )
      (3): ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=150, bias=True)
        )
      )
      (4): ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=512, bias=True)
        )
      )
      (5): ProjectionHead(
        (head): Sequential(
          (linear_final): L

In [5]:
# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hypernet = hypernet.to(device)
mainnet = mainnet.to(device)

In [6]:
# Load the kinematic chain for FK calculations (optional)
r_arm = ikpy.chain.Chain.from_urdf_file(cfg.chain_path)

# Extract joint limits
upper = []
lower = []
for i in range(1, len(r_arm.links) - 1):
    lower.append(r_arm.links[i].bounds[0])
    upper.append(r_arm.links[i].bounds[1])

upper = np.array(upper)
lower = np.array(lower)


c:\Users\SAE\Desktop\Organisatorisches\FH\AI\3. Semester\robotics_project\.venv\Lib\site-packages\ikpy\chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))
c:\Users\SAE\Desktop\Organisatorisches\FH\AI\3. Semester\robotics_project\.venv\Lib\site-packages\ikpy\chain.py:60: UserWarning: Link ee_fixed_joint (index: 7) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transfor

In [7]:
# Example input position (e.g., desired end-effector position)
positions = torch.tensor([[0.5, 0.2, 0.3, 0, 0, 0]], dtype=torch.float32)  # Replace with your input positions
positions = positions.to(device)


In [8]:
# Predict weights using HyperNet
with torch.no_grad():  # Disable gradient computation
    predicted_weights = hypernet(positions)

# Generate joint angles using MainNet
with torch.no_grad():
    initial_input = torch.ones((positions.shape[0], 1), dtype=torch.float32).to(device)
    samples, distributions, means, variance, selection = mainnet.validate(
        initial_input, predicted_weights, lower, upper
    )

# Convert the predicted joint angles to a readable format
predicted_joint_angles = []
for sample in samples:
    predicted_joint_angles.append([angle.item() for angle in sample])


In [9]:
predicted_joint_angles

[[0.2704930603504181],
 [-0.3294661045074463],
 [-0.9687324166297913],
 [1.6628884077072144],
 [-0.45977580547332764],
 [0.6450687050819397]]

In [10]:
# Flatten the predicted joint angles
flat_joint_angles = [angle[0] for angle in predicted_joint_angles]


In [11]:
# Add base and end-effector placeholders (if necessary)
full_joint_angles = [0] + flat_joint_angles + [0]  # Base and end-effector placeholders

In [12]:
full_joint_angles

[0,
 0.2704930603504181,
 -0.3294661045074463,
 -0.9687324166297913,
 1.6628884077072144,
 -0.45977580547332764,
 0.6450687050819397,
 0]

In [13]:
len(r_arm.links) 

8

In [14]:


# Check for length mismatch
if len(full_joint_angles) != len(r_arm.links):
    raise ValueError("Mismatch between joint angles and kinematic chain.")

# Compute Forward Kinematics
fk_position = r_arm.forward_kinematics(full_joint_angles)[:3, 3]  # Extract end-effector position

In [15]:

print(f"FK Position using predicted joints: {fk_position}")
print(f"original position: {positions[0]}")


FK Position using predicted joints: [0.3750051  0.29378955 0.52902873]
original position: tensor([0.5000, 0.2000, 0.3000, 0.0000, 0.0000, 0.0000], device='cuda:0')
